In [2]:
#-------------------------! import statements !-------------------------
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import BartTokenizer, BartForConditionalGeneration
import time
from dataclasses import dataclass

#-------------------------! Hyperparameter !-------------------------
@dataclass
class BartConfig:
    model_name: str = "facebook/bart-base"
    batch_size: int = 2
    block_size: int = 1024 

#-------------------------! Load BART Model and Tokenizer !-------------------------
config = BartConfig()
tokenizer = BartTokenizer.from_pretrained(config.model_name)
model = BartForConditionalGeneration.from_pretrained(config.model_name)

device = "cpu"
if torch.cuda.is_available():
    device = "cuda"
elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
    device = "mps"
print(f'using device {device}')

model.to(device)
model = torch.compile(model, backend='eager')

optimizer = optim.AdamW(model.parameters(), lr=1e-4)

#-------------------------! Data Loader Class !-------------------------
class DataLoaderLite:
    def __init__(self, B, T, tokenizer):
        self.B = B
        self.T = T
        self.tokenizer = tokenizer

        with open('../input.txt', 'r') as f:
            text = f.read()
        tokens = tokenizer(text, return_tensors="pt", max_length=T, truncation=True, padding="max_length")["input_ids"]
        self.tokens = tokens.to(device)

        self.num_batches = max(len(self.tokens) // B, 1)
        print(f'1 epoch = {self.num_batches} batches')

        # state
        self.current_position = 0

    def next_batch(self):
        B, T = self.B, self.T
        start_idx = self.current_position
        end_idx = start_idx + B

        x = self.tokens[start_idx:end_idx]
        y = self.tokens[start_idx:end_idx]

        self.current_position += B

        if self.current_position + B > len(self.tokens):
            self.current_position = 0

        return x, y

train_loader = DataLoaderLite(config.batch_size, config.block_size, tokenizer)

def train(model, train_loader, optimizer, num_epochs=100):
    model.train()
    for i in range(num_epochs):
        t0 = time.time()
        src, tgt = train_loader.next_batch()
        optimizer.zero_grad()
        with torch.autocast(device_type=device, dtype=torch.bfloat16):
            output = model(input_ids=src, labels=tgt)
            loss = output.loss

        loss.backward()
        optimizer.step()
        torch.cuda.synchronize()
        t1 = time.time()
        dt = (t1 - t0) * 1000  # time difference is in milliseconds
        tokens_per_sec = (train_loader.B * train_loader.T) / (t1 - t0)
        if (i + 1) % 10 == 0:
            print(f'step {i + 1}, loss: {loss.item()}, dt: {dt:.2f} ms, tok/sec {tokens_per_sec}')

train(model, train_loader, optimizer, num_epochs=100)


c:\Users\PARIKSHIT\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

c:\Users\PARIKSHIT\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\PARIKSHIT\.cache\huggingface\hub\models--facebook--bart-large. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

using device cpu
1 epoch = 0 batches


KeyboardInterrupt: 